In [61]:
# импортируем необходимые модули
import pandas as pd
import numpy as np

In [62]:
# считываем данные
df = pd.read_csv('data/data.csv')

In [63]:
# выясним размерность датасета
print(df.shape)
# проверим корректность загрузки и ознакомимся с полями
df.head()

(377185, 18)


,status,private pool,propertyType,street,baths,homeFacts,fireplace,city,schools,sqft,zipcode,beds,state,stories,mls-id,PrivatePool,MlsId,target
0,Active,NaN,Single Family Home,240 Heather Ln,3.5,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",Gas Logs,Southern Pines,"[{'rating': ['4', '4', '7', 'NR', '4', '7', 'N...",2900,28387,4,NC,NaN,NaN,NaN,611019,"$418,000"
1,for sale,NaN,single-family home,12911 E Heroy Ave,3 Baths,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",NaN,Spokane Valley,"[{'rating': ['4/10', 'None/10', '4/10'], 'data...","1,947 sqft",99216,3 Beds,WA,2.0,NaN,NaN,201916904,"$310,000"
2,for sale,NaN,single-family home,2005 Westridge Rd,2 Baths,"{'atAGlanceFacts': [{'factValue': '1961', 'fac...",yes,Los Angeles,"[{'rating': ['8/10', '4/10', '8/10'], 'data': ...","3,000 sqft",90049,3 Beds,CA,1.0,NaN,yes,FR19221027,"$2,895,000"
3,for sale,NaN,single-family home,4311 Livingston Ave,8 Baths,"{'atAGlanceFacts': [{'factValue': '2006', 'fac...",yes,Dallas,"[{'rating': ['9/10', '9/10', '10/10', '9/10'],...","6,457 sqft",75205,5 Beds,TX,3.0,NaN,NaN,14191809,"$2,395,000"
4,for sale,NaN,lot/land,1524 Kiscoe St,NaN,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",NaN,Palm Bay,"[{'rating': ['4/10', '5/10', '5/10'], 'data': ...",NaN,32908,NaN,FL,NaN,NaN,NaN,861745,"$5,000"


In [64]:
# оценим количество пропусков и типы данных
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 377185 entries, 0 to 377184
Data columns (total 18 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   status        337267 non-null  object
 1   private pool  4181 non-null    object
 2   propertyType  342452 non-null  object
 3   street        377183 non-null  object
 4   baths         270847 non-null  object
 5   homeFacts     377185 non-null  object
 6   fireplace     103115 non-null  object
 7   city          377151 non-null  object
 8   schools       377185 non-null  object
 9   sqft          336608 non-null  object
 10  zipcode       377185 non-null  object
 11  beds          285903 non-null  object
 12  state         377185 non-null  object
 13  stories       226470 non-null  object
 14  mls-id        24942 non-null   object
 15  PrivatePool   40311 non-null   object
 16  MlsId         310305 non-null  object
 17  target        374704 non-null  object
dtypes: object(18)
memory usa

In [65]:
# посмотрим на количество пропусков с более удобного ракурса
df.isnull().sum()

status           39918
private pool    373004
propertyType     34733
street               2
baths           106338
homeFacts            0
fireplace       274070
city                34
schools              0
sqft             40577
zipcode              0
beds             91282
state                0
stories         150715
mls-id          352243
PrivatePool     336874
MlsId            66880
target            2481
dtype: int64

Описание данных:

- 'status' — статус продажи;
- 'private pool' и 'PrivatePool' — наличие собственного бассейна;
- 'propertyType' — тип объекта недвижимости;
- 'street' — адрес объекта;
- 'baths' — количество ванных комнат;
- 'homeFacts' — сведения о строительстве объекта (содержит несколько типов сведений, влияющих на оценку объекта);
- 'fireplace' — наличие камина;
- 'city' — город;
- 'schools' — сведения о школах в районе;
- 'sqft' — площадь в футах;
- 'zipcode' — почтовый индекс;
- 'beds' — количество спален;
- 'state' — штат;
- 'stories' — количество этажей;
- 'mls-id' и 'MlsId' — идентификатор MLS (Multiple Listing Service, система мультилистинга);
- 'target' — цена объекта недвижимости (целевой признак, который необходимо спрогнозировать).

In [66]:
# проверим пару признаков относительно частного бассейна
# для начала проверим варианты внесения информации
display(df['private pool'].value_counts())
display(df['PrivatePool'].value_counts())

Yes    4181
Name: private pool, dtype: int64

yes    28793
Yes    11518
Name: PrivatePool, dtype: int64

In [67]:
# предположим, что это признаки, созданные в базе данных в разное время
# т.е., информация о наличии бассейна есть либо в одном, либо в другом
# в таком случае, не должно быть записей, в которых одновременно указано наличие в обоих полях
# проверим простым способом - переведем наличие в "1" и сложим оба столбца
# если запись и там, и там, получим в этих записях двойки

# для начала переведем записи в единицы и проверим сохранение количеств значений
df['PrivatePool'] = df['PrivatePool'].apply(lambda x: 1 if x in ['yes', 'Yes'] else 0)
display(df['PrivatePool'].value_counts())
df['private pool'] = df['private pool'].apply(lambda x: 1 if x in ['yes', 'Yes'] else 0)
display(df['private pool'].value_counts())

0    336874
1     40311
Name: PrivatePool, dtype: int64

0    373004
1      4181
Name: private pool, dtype: int64

In [68]:
# поскольку перекодирование столбцов прошло успешно, теперь делаем аггрегирующее поле
# проверка вариантов значений покажет, подтвердилось ли вышеописанное предположение
df['PoolPrivate'] = df['PrivatePool'] + df['private pool']
df['PoolPrivate'].value_counts()

0    332693
1     44492
Name: PoolPrivate, dtype: int64

In [69]:
# схема сработала, изначальные признаки можно удалить
df = df.drop(columns=['private pool', 'PrivatePool'], axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 377185 entries, 0 to 377184
Data columns (total 17 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   status        337267 non-null  object
 1   propertyType  342452 non-null  object
 2   street        377183 non-null  object
 3   baths         270847 non-null  object
 4   homeFacts     377185 non-null  object
 5   fireplace     103115 non-null  object
 6   city          377151 non-null  object
 7   schools       377185 non-null  object
 8   sqft          336608 non-null  object
 9   zipcode       377185 non-null  object
 10  beds          285903 non-null  object
 11  state         377185 non-null  object
 12  stories       226470 non-null  object
 13  mls-id        24942 non-null   object
 14  MlsId         310305 non-null  object
 15  target        374704 non-null  object
 16  PoolPrivate   377185 non-null  int64 
dtypes: int64(1), object(16)
memory usage: 48.9+ MB


In [70]:
# идентификаторы MLS в двух ипостасях не несут в себе описательной информации применительно к объекту
# по этой причине удаляем их

# df = df.drop(columns=['mls-id', 'MlsId'], axis=1)
# df.info()

In [71]:
# теперь проанализируем датасет на наличие полных дубликатов
df.duplicated().sum()

50

In [72]:
# удалим дубликаты и проверим итоговую размерность датасета, высота должна получиться 377185-141=377044
df = df.drop_duplicates()
df.shape

(377135, 17)